In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries
import numpy as np
import cv2
import os
import pandas as pd
from PIL import Image
import keras 
np.random.seed(1000)
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Model,Sequential, Input, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications import DenseNet121

In [ ]:
import os
import glob
import math
from PIL import Image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical

In [ ]:
#数据路径
data_dir = "../input/plant-pathology-2021-fgvc8"

#获取指定目录下的所有图片
train_image_path = glob.glob(os.path.join(data_dir, "train_images/*.jpg"))

#读取数据csv文件
label_df = pd.read_csv(os.path.join(data_dir, "train.csv"))

# Number od trainin images
print("Number of training images: {}".format(len(train_image_path)))

In [ ]:
#返回列的所有唯一值（labels）
labels = label_df.labels.unique()
print(labels)
print(len(labels))

In [ ]:
# Count number of data for each labels
label_count = label_df.labels.value_counts()
print(label_count)
label_ratio = label_df.labels.value_counts(normalize=True, sort=True)
print(label_ratio)

In [ ]:
train_dir= '../input/plant-pathology-2021-fgvc8/train_images'
test_dir =  '../input/plant-pathology-2021-fgvc8/test_images'
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train


In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv' ,dtype=str)
train.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
# 将类别数据数字化
train['labels_type'] = labelencoder.fit_transform(train['labels'])
train

In [ ]:
df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv' ,dtype=str)
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df['labels_type'] = labelencoder.fit_transform(df['labels'])
df

In [ ]:
ref = df.labels.unique()
ref

In [ ]:
df.labels_type.unique()

In [ ]:
rng = 42
train_csv = df.sample(frac=0.8, random_state=rng)
test_csv = df.loc[~df.index.isin(train_csv.index)]
train_csv.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_augmenter=ImageDataGenerator(
    rescale=1./255, 
    #rotation range and fill mode only
    samplewise_center=True, 
    samplewise_std_normalization=True, 
    horizontal_flip = True, 
    vertical_flip = True, 
    height_shift_range= 0.05, 
    width_shift_range=0.1, 
    rotation_range=45, 
    shear_range = 0.1,
    fill_mode = 'nearest',
    zoom_range=0.10,
    #preprocessing_function=function_name,
    )

test_augmenter=ImageDataGenerator(
    rescale=1./255
    )
train_csv

In [ ]:

IMAGE_SIZE = 64
def read_image(filepath):
    return cv2.imread(os.path.join(data_dir, filepath))
#调整图像到目标大小
def resize_image(image, image_size):
    return cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_AREA)

In [ ]:
Image_path = '/kaggle/input/plant-pathology-2021-fgvc8/train_images'
batch_size=16
IMG_size=64
train_generator=train_augmenter.flow_from_dataframe(
dataframe=train_csv,
directory=Image_path,
#save_to_dir='augmented',
#save_prefix='_aug'
#save_format='jpg'
x_col='image',
y_col='labels',
batch_size=batch_size,
seed=42,
shuffle=True,
class_mode='categorical',
target_size=(64, 64),
)

In [ ]:
# image path for the testing dataset
Image_path = '/kaggle/input/plant-pathology-2021-fgvc8/train_images'
test_generator=test_augmenter.flow_from_dataframe(
dataframe=test_csv,
directory=Image_path,
#save_to_dir='augmented',
#save_prefix='_aug'
#save_format='jpg'
x_col='image',
y_col='labels',
batch_size=batch_size,
seed=42,
class_mode='categorical',
target_size=(64, 64),
)

In [ ]:
INPUT_SHAPE = (64, 64, 3)
def build_model():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                     input_shape=INPUT_SHAPE))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    #model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12, activation='softmax'))

    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model = build_model()
history = model.fit(train_generator,epochs=4, steps_per_epoch = 190)

In [ ]:
model.save('model.h5')

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import glob
test = glob.glob('/kaggle/input/plant-pathology-2021-fgvc8/test_images/' +'*'+'.jpg')
test

In [ ]:
sample = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv')
sample['labels'][0]

In [ ]:
def load_real_samples(X1):
    X1 = X1/ 255
    return [X1]

In [ ]:
from PIL import Image
for i in range(len(test)):
    inp = load_img(test[i])
    img =inp.resize(( 64, 64), Image.ANTIALIAS)
    gh1 = img_to_array(img)
    ph = load_real_samples(gh1)
    ph1 = np.array(ph)
    pred = model.predict(ph1)
    predc = ref[np.argmax(pred[0,:])]
    sample['labels'][i] = predc
    print(predc)

In [ ]:
sample.to_csv('submission.csv', index = False)